In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = None
# Set up folders and time variables
tday = datetime.now().strftime("%d-%m-%Y")
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
folder = r"\\lawoffice\GSLODocuments\LegalServices_Division\01.Lawoffice_Common\BUSINESS ANALYSTS\BI JIRA\jira_app"
from jira_helper import jira_connect,format_date,extract_year,extract_month,minutes
df = jira_connect()
len(df)
# df.sort_values(by='Issue Key',ascending=True).head(5)

3166

In [ ]:

mapping_file = r"\\lawoffice\GSLODocuments\LegalServices_Division\01.Lawoffice_Common\BUSINESS ANALYSTS\BI JIRA\mapping names jira.xlsx"
mapping_df = pd.read_excel(mapping_file)
mapping_dict = mapping_df.set_index('Raw')['Cleaned'].to_dict()
dept_dict = mapping_df.set_index('Raw')['Department'].to_dict()
company_dict = mapping_df.set_index('Raw')['Company'].to_dict() 

df['Company'] = df['Reporter'].map(company_dict)
df['DNY_Assignee'] = df['Assignee'].map(company_dict)
df['Department'] = df['Reporter'].map(dept_dict)
df= df.replace(mapping_dict)
df['Assignee'] = np.where((df['Assignee'].astype(str) == '0') & (df['Reporter'] == 'Βαγιάννης'), 'Βαγιάννης', 
                 np.where((df['Assignee'].astype(str) == '0') & (df['Reporter'] == 'Παπαδόπουλος'), 'Παπαδόπουλος', df['Assignee']))

test = df
# test = test.dropna(subset='Issue key') @extra dupes1
# # Replace null values with 0
test.fillna(0, inplace=True)
# # 1) Μετατροπή ημερομηνίας σε date format (απαιτεί και την ρύθμιση ως ημερομηνίας εντός του bi)
format_date('Created',test)
# format_date('Updated',test)
format_date('Resolved',test)
print(test.head())
# # 2) Ομαδοποίηση Status σε κατηγορίες Εκκρεμή και Μη Εκκρεμή
pending_list = ['Open', 'In Progress', 'Approved', 'For Approval', 'Specification Analysis','Reopened']
test['Εκκρεμή'] = np.where(test['Status'].isin(pending_list), "Εκκρεμή", "Μη Εκκρεμή")
# # 3) Rpa and Iteration Flag
test['RPA_Flag'] = np.where(df['Summary'].str.contains('rpa', case=False, na=False), 'RPA', 'No-RPA')
test['Iteration_Flag'] = np.where(df['Label'] == '#C.It', 'Iteration', 'No-Iteration')

# test.to_excel(os.path.join(folder,'test.xlsx'))
# os.startfile('test.xlsx')
#  # 4) Υπολογισμός Αρχικής Εκτίμησης
test['Estimate'] = test['Estimate'] / 3600
test['Estimate'] = test['Estimate'].astype(float).apply(lambda x : round(x,2))


# Get the non log columns
cols = ['Issue Key',
       'Summary',
       'Status',
       'Created',
       'Resolved',
       'Assignee',
       'Reporter',
       'Updated_date',
       'Created_date',
       'Resolved_date',
       'Original estimate',
       'RPA_Flag',
       'Iteration_Flag',
       'Εκκρεμή',
       'Estimate',
       'Company',
       'DNY_Assignee',
       'Department']

# Get the log columns
log_cols = [f'Log Work.{i}' if i > 0 else 'Log Work' for i in range(99)]
hours_cols = ['Issue Key', 'hours_assignee','before','during','after','Most_recent_update','Log_Month','Log_Year']# 'Time_before_resolved', 'Time_in_resolved','Time_after_resolved',


def melt_df(df) :
    melted = pd.melt(df,id_vars=cols,value_vars=log_cols,value_name='Logs')
    melted['Updated_Month'] = melted['Updated_date'].dt.month.astype(str)
    melted['Updated_date'] = pd.to_datetime(melted['Updated_date'], errors='coerce')
    melted['Log_Month'] = melted['Logs'].apply(extract_month).fillna("0")
    melted['Log_Year'] = melted['Logs'].apply(extract_year).fillna("0")
    melted['Log_Year'] = melted['Logs'].apply(extract_year)
    melted['Resolved_Month'] = melted['Resolved'].apply(extract_month)
    melted['Resolved_Year'] = melted['Resolved'].apply(extract_year)
    melted['minutes_assignee'] = melted['Logs'].apply(minutes)
    melted['minutes_assignee'] = melted['minutes_assignee'].fillna(0).astype(int)
    melted['hours'] = melted['minutes_assignee']  / 3600
    melted['hours'] = melted['hours'].astype(float).round(2)
    before = (melted['Log_Month'] < melted['Resolved_Month']) & (melted['Log_Year'] <= melted['Resolved_Year']) & (melted['Status'].isin(['Resolved','Closed']))
    during = (melted['Log_Month'] == melted['Resolved_Month']) & (melted['Log_Year'] == melted['Resolved_Year']) & melted['Status'].isin(['Resolved','Closed'])
    after = (melted['Log_Month'] > melted['Resolved_Month']) &  (melted['Log_Year'] >= melted['Resolved_Year']) &melted['Status'].isin(['Resolved','Closed'])
    # Calculate time spent before the resolution month, handling missing values
    melted['Time_before_resolved'] = melted[before].groupby('Issue key')['hours'].transform('sum').fillna(0)
    # Calculate time spent in the resolved month, handling missing values
    melted['Time_in_resolved'] = melted[during].groupby('Issue key')['hours'].transform('sum').fillna(0)
    # Calculate time spent after the resolution month
    melted['Time_after_resolved'] = melted[after].groupby('Issue key')['hours'].transform('sum').fillna(0)
    # Calculate the total hours spent on the issue by assignee
    melted['hours_assignee'] = melted.groupby('Issue key')['hours'].transform('sum')
    return melted


# print("Unpivoting df")
# # # Unpivot to get logs as rows
hours_df = melt_df(test)
# print("Unpivoted df")
hours_df['before'] = hours_df.groupby('Issue Key')['Time_before_resolved'].transform('min')
hours_df['after'] = hours_df.groupby('Issue Key')['Time_after_resolved'].transform('min')
hours_df['during'] = hours_df.groupby('Issue Key')['Time_in_resolved'].transform('min')
hours_df['Most_recent_update'] = hours_df.groupby('Issue key')['Updated_date'].transform('max')
hours_df['Most_recent_update'] = pd.to_datetime(hours_df['Most_recent_update'],format='%d/%m/%Y',errors='coerce').dt.strftime('%d/%m/%Y')
# # hours_df = hours_df.drop_duplicates() @extra dupes2
hours_df.to_excel(os.path.join(folder,'hours_df.xlsx'))
os.startfile('hours_df.xlsx')



c:\Users\pallist\Desktop\Desktop\ΤΡΕΧΟΝΤΑ\Code November 2024\Large Scripts\jira_helper.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"_date"] = pd.to_datetime(df[column], errors='coerce', dayfirst=True)
c:\Users\pallist\Desktop\Desktop\ΤΡΕΧΟΝΤΑ\Code November 2024\Large Scripts\jira_helper.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"_date"] = pd.to_datetime(df[column], errors='coerce', dayfirst=True)
C:\Users\pallist\AppData\Local\Temp\ipykernel_26408\1800153737.py:25: PerformanceWarning: Data

  Issue Key   
0  DNY-3236  \
1  DNY-3235   
2  DNY-3234   
3  DNY-3233   
4  DNY-3232   

                                                                    Summary   
0                                       DNY - 2870 Do Value Άμεσοι έλεγχοι   \
1                                  ΠΡΟΣΘΗΚΗ ΥΠΟΘΕΣΕΩΝ ΣΤΗΝ ΑΝΑΦΟΡΑ ΑΙΤ-3387   
2                                  Επανέλεγχοι για προσημείωση, απο Κομίλη.   
3                           Μαζική εισαγωγή πληρωμών (κτηματολόγιο) στο CMS   
4  Ενημέρωση mappings RPA Επιταγών Α Πειραιώς και λεκτικού επιταγής Εθνικής   

   Reporter      Assignee    Status                       Created   
0    Πανίκα        Κοσμής      Open  2024-12-11T12:51:54.858+0200  \
1  Κανναβού        Βούρας      Open  2024-12-11T12:50:50.018+0200   
2   Μποζώνη       Μποζώνη  Resolved  2024-12-11T12:48:04.589+0200   
3    Πανίκα       Ρούσσου      Open  2024-12-11T12:05:44.908+0200   
4    Πάλλης  Παπαδόπουλος      Open  2024-12-11T11:55:11.423+0200   

           Updated 

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# hours_df = hours_df.drop_duplicates(subset='Issue key')
# hours_df = hours_df[hours_cols]
# test = test[cols]
# merged_df = pd.merge(hours_df,test,on='Issue key',how='inner')
# print("Merged df")
# merged_df['Απόκλιση'] = merged_df['hours_assignee'] - merged_df['Estimate'] #processing
# # merged_df['Απόκλιση'] = merged_df['Απόκλιση'].apply(lambda x : round(x,2)) 
# merged_df['Εντός Εκτίμησης'] = np.where(merged_df['Απόκλιση'] <= 0, 'Εντός Εκτίμησης', 'Εκτός Εκτίμησης')#processing
# merged_df['Εντός Εκτίμησης'] = np.where(merged_df['Estimate'] == 0, 'Δεν έχει δοθεί εκτίμηση', merged_df['Εντός Εκτίμησης'])#processing
# merged_df['Created_Resolved_days'] = np.where(
#     (merged_df['Status'] == 'Resolved') | (merged_df['Status'] == 'Closed'),
#     (merged_df['Resolved_date'] - merged_df['Created_date']).dt.days,
#     np.nan)
# today = pd.Timestamp(datetime.today())
# merged_df['Created_vs_Today_Days'] = (today - merged_df['Created_date']).dt.days
# merged_df['Key'] = merged_df['Issue key'].str.strip("DNY-").astype(int)
# merged_df['Created_Today'] = np.where((merged_df['Created_date'].dt.date == today.date()) & (merged_df['Created_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Created_Current_Week'] = np.where((merged_df['Created_date'].dt.isocalendar().week == today.isocalendar().week) & (merged_df['Created_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Created_Last_Week'] = np.where(merged_df['Created_date'].dt.isocalendar().week == (today.isocalendar().week - 1), 'Yes', 'No')
# merged_df['Created_Current_Month'] = np.where((merged_df['Created_date'].dt.month == today.month) & (merged_df['Created_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Resolved_Today'] = np.where((merged_df['Resolved_date'].dt.date == today.date()) & (merged_df['Resolved_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Resolved_Current_Week'] = np.where((merged_df['Resolved_date'].dt.isocalendar().week == today.isocalendar().week) & (merged_df['Resolved_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Resolved_Last_Week'] = np.where(merged_df['Resolved_date'].notna() & (merged_df['Resolved_date'].dt.isocalendar().week == (today.isocalendar().week - 1)), 'Yes', 'No')
# merged_df['Resolved_Current_Month'] = np.where((merged_df['Resolved_date'].dt.month == today.month) & (merged_df['Resolved_date'].dt.year == today.year), 'Yes', 'No')
# merged_df['Week_Resolved'] = merged_df['Resolved_date'].dt.isocalendar().week
# merged_df['Week_Created'] = merged_df['Created_date'].dt.isocalendar().week

# delete_old_files(folder)
# merged_df.to_excel(os.path.join(folder,'data_for_bi.xlsx'))
# # os.startfile(os.path.join(folder,'data_for_bi.xlsx'))




# import openpyxl
# from openpyxl import load_workbook
# from openpyxl.styles import Font, PatternFill, Alignment

# def format_df(file) :
#     workbook = openpyxl.load_workbook(file)
#     for worksheet in workbook:
#         font = Font(color='FFFFFF', bold=True)
#         fill = PatternFill(start_color='5552A2', end_color='5552A2', fill_type='solid')
#         alignment = Alignment(horizontal='left')  # Left align
#         for cell in worksheet[1]:
#             cell.font = font
#             cell.fill = fill
#             cell.alignment = alignment
#         for column in worksheet.columns:
#             worksheet.column_dimensions[openpyxl.utils.get_column_letter(column[0].column)].width = 40      

#     workbook.save(file)

# clean_df = merged_df
# clean_df.rename(columns={'before': 'Hours_Spent_Before_Month_of_Resolution', 
#                          'during': 'Hours_Spent_In_Month_of_Resolution',
#                          'after' : 'Hours_Spent_After_Month_Of_Resolution',
#                          'Created_Resolved_days': 'Days_Between_Creation_and_Resolution',
#                          'Created_vs_Today_Days': 'Days_Since_Creation',
#                          'Απόκλιση': 'Hour_Difference_From_Estimate',
#                          'hours_assignee': 'Total_Hours_Spent',                         
#                          }, inplace=True)

# # clean_df.drop(columns=['Original estimate','Updated_date','Created','Resolved'], inplace=True)
# clean_df.sort_values(by='Key',inplace=True)
# clean_df['Resolved_date'] = pd.to_datetime(clean_df['Resolved_date'],format='%d/%m/%Y',errors='coerce').dt.strftime('%d/%m/%Y')
# clean_df['Created_date'] = pd.to_datetime(clean_df['Created_date'],format='%d/%m/%Y',errors='coerce').dt.strftime('%d/%m/%Y')
# clean_df['Most_recent_update'] = pd.to_datetime(clean_df['Most_recent_update'],format='%d/%m/%Y',errors='coerce').dt.strftime('%d/%m/%Y')
# clean_df.to_excel(os.path.join(folder,'Excel_Report_Jira.xlsx'),index=False)
# format_df(os.path.join(folder,'Excel_Report_Jira.xlsx'))
# os.startfile(os.path.join(folder,'Excel_Report_Jira.xlsx'))
# end_time = datetime.now()
# print(f"Operation complete : {start_time.strftime('%H:%M:%S')} - {end_time.strftime('%H:%M:%S')}")




